<a href="https://colab.research.google.com/github/mwvgroup/Pitt-Google-Broker/blob/u%2Ftjr%2Ftutorials/tutorials/PGB_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1) Data Products

Architecture figure highlighting the data products available to them

Links to webpages:
- [BQ table descriptions and schemas]
- [Cloud Storage buckets]
- [Pub/Sub streams]

---

# 2) Setup

- Create GCP Project
- Enable billing
- Enable APIs

In [ ]:
#--- Colab Setup
from google.colab import auth
auth.authenticate_user()  # follow the instructions to authorize Google Cloud SDK 

# connect your file system (local or Drive)
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Colab Notebooks/'
# example of down/uploading a file (also see file browser on left)


#--- Cloud Shell Setup


#--- Local machine setup


In [ ]:
from google.cloud import bigquery
from google.cloud import storage


In [ ]:
my_project_id = 'ardent-cycling-243415'
pgb_project_id = 'ardent-cycling-243415'

---

# 3) Query a BigQuery Database

- [Query pricing](https://cloud.google.com/bigquery/docs/query-overview#query_pricing) (charges based on number of bytes processed; first 1 TB of data processed per month, per billing account, is __free__.)

## 3a) Python

standard SQL default ([change to legacy SQL](https://cloud.google.com/bigquery/docs/reference/standard-sql/enabling-standard-sql#changing_from_the_default_dialect))

Links with more info:
- [Documentation](https://googleapis.dev/python/bigquery/latest/index.html)
- [Colab Snippets](https://colab.research.google.com/notebooks/snippets/bigquery.ipynb#scrollTo=jl97S3vfNHdz)

In [ ]:
# create a connection to BigQuery
bq_client = bigquery.Client(project=my_project_id)

# write the SQL query as a string
sql = """
    SELECT name, SUM(number) as count
    FROM `bigquery-public-data.usa_names.usa_1910_current`
    GROUP BY name
    ORDER BY count DESC
    LIMIT 10
"""

# make an API request
query_job = bq_client.query(sql)

In [ ]:
# Option1: dump results to a pandas.DataFrame
df = query_job.to_dataframe()
df

In [ ]:
# Option 2: parse results row by row
print("The query data:")
for row in query_job:
    # row values can be accessed by field name or index
    print(f"name={row[0]}, count={row['count']}")

## 3b) Command-line tool `bq`

legacy SQL default ([change to standard SQL](https://cloud.google.com/bigquery/docs/reference/standard-sql/enabling-standard-sql#changing_from_the_default_dialect))

Links with more info:
- [Running queries from the bq command-line tool](https://cloud.google.com/bigquery/docs/bq-command-line-tool#running_queries_from_the) (also see other sections on this page)



```bash
#--- retrieve query results from command line
# (this cell is not executable)

bq query --use_legacy_sql=false \
'SELECT
  word,
  SUM(word_count) AS count
FROM
  `bigquery-public-data`.samples.shakespeare
WHERE
  word LIKE "%raisin%"
GROUP BY
  word'
```



---

# 4) Download a file from Cloud Storage

- grab cutouts from Cloud Storage


In [ ]:
#--- Download an alert from GCS bucket

import logging
import os

bucket_name = f'{pgb_project_id}_ztf_alert_avro_bucket'
fname_prefix = '1605062619785'  # recently ingested avro file name, get from logging
local_dir = './testdownload'
delimiter = '/'

storage_client = storage.Client.from_service_account_json('GCPauth_pitt-google-broker-prototype-0679b75dded0.json')
bucket = storage_client.get_bucket(bucket_name)
blobs = bucket.list_blobs(prefix=fname_prefix, delimiter=delimiter) #List all objects that satisfy the filter.
# Download the file to a destination
# Iterating through for loop one by one using API call
for blob in blobs:
    destination_uri = '{}/{}'.format(local_dir, blob.name)
    blob.download_to_filename(destination_uri)

---

# 5) Listen to Pub/Sub Streams

- Python
  - unpack into dict

---

# 6) BigQuery ML

- [What is BigQuery ML?](https://cloud.google.com/bigquery-ml/docs/introduction)
- [Quickstart using the Cloud Console](https://cloud.google.com/bigquery-ml/docs/bigqueryml-web-ui-start)
- [Applying Machine Learning to your Data with GCP](https://www.coursera.org/learn/data-insights-gcp-apply-ml)

---

# 7) Filter or Process the Data (using Apache Beam)

We use the [Apache Beam](https://beam.apache.org/) (Python) SDK to write the data pipeline (see also [Apache Beam Programming Guide](https://beam.apache.org/documentation/programming-guide/)). This has the following advantages:

1. Convenience: Native I/O functions for Pub/Sub, BigQuery, and Cloud Storage.
2. Flexiblity: The same pipeline can accept streaming and batch inputs. -> Use the same pipeline to process the live stream and reprocess the database.
3. Portability: The same pipeline can be run/executed in multiple environments (Google Cloud, AWS, local machine) via an execution "runner" ([Apache Flink](https://beam.apache.org/documentation/runners/flink/), [Apache Spark](https://beam.apache.org/documentation/runners/spark/), [Google Dataflow](https://beam.apache.org/documentation/runners/dataflow/), or [direct/local](https://beam.apache.org/documentation/runners/direct/)).

Links with more info:
- [Colab Snippets: Apache Beam](https://colab.research.google.com/github/apache/beam/blob/master/examples/notebooks/get-started/try-apache-beam-py.ipynb)

In [ ]:
#--- Install Apache Beam (and other setup)

# Run and print a shell command.
def run(cmd):
  print('>> {}'.format(cmd))
  !{cmd}
  print('')

# Install apache-beam.
run('pip install --quiet apache-beam')

# Copy the input file into the local file system.
run('mkdir -p data')
run('gsutil cp gs://dataflow-samples/shakespeare/kinglear.txt data/')

In [ ]:
#--- Minimal word count (example)

import apache_beam as beam
import re

inputs_pattern = 'data/*'
outputs_prefix = 'outputs/part'

# Running locally in the DirectRunner.
with beam.Pipeline() as pipeline:
  (
      pipeline
      | 'Read lines' >> beam.io.ReadFromText(inputs_pattern)
      | 'Find words' >> beam.FlatMap(lambda line: re.findall(r"[a-zA-Z']+", line))
      | 'Pair words with 1' >> beam.Map(lambda word: (word, 1))
      | 'Group and sum' >> beam.CombinePerKey(sum)
      | 'Format results' >> beam.Map(lambda word_count: str(word_count))
      | 'Write results' >> beam.io.WriteToText(outputs_prefix)
  )

# Sample the first 20 results, remember there are no ordering guarantees.
run('head -n 20 {}-00000-of-*'.format(outputs_prefix))

In [ ]:

import apache_beam as beam


In [ ]:
# Functions and classes that will be packaged into a module 
# and hidden from the user.

class extractAlertDict(beam.DoFn):
    def process(self, msg):
        from io import BytesIO
        from fastavro import reader

        # Extract the alert data from msg -> [dict]
        with BytesIO(msg) as fin:
            alertDicts = [r for r in reader(fin)]  # contains 1 dict

        return alertDicts




In [ ]:
# Functions and classes for the user to experiment with.

#--- Filters
# https://beam.apache.org/documentation/transforms/python/elementwise/filter/

# example filter for likely solar system objects
def nearby_ssobject(alert):
    candidate = alert['candidate']
    ssobject_is_near = (candidate['ssdistnr'] > 0) and (candidate['ssdistnr'] < 5)
    return ssobject_is_near

# write your own filter
def my_filter(alert):
    """
    Args:
        alert (dict): a single alert
    Returns:
        keep_alert (bool): True to keep the alert, False to filter it out
    """


#--- Other Functions (arbitrary processing)
# 

class my_function(beam.DoFn):
    def process(self, alertDict):

        # Process the alert however you want
        # here we simply extract the object and candidate ids
        oid, cid = alertDict['objectId'], alertDict['candid']
        ids = {'objectId': oid, 'candid': cid}

        # Return your output in a list
        return [ids]


In [ ]:
# Run a Beam pipeline

topic = 'ztf_alert_data-tjr'

with beam.Pipeline(options=pipeline_options) as pipeline:
    (
        pipeline
        # read Pub/Sub messages (bytes) into the pipeline
        | 'ReadFromPubSub' >> beam.io.ReadFromPubSub(topic=topic)
        # extract the alert bytes into a dict
        | 'ExtractAlertDict' >> beam.ParDo(extractAlertDict()))
        # filter, keep alerts likely to be a solar system object
        | 'nearby_ssobject' >> beam.Filter(nearby_ssobject)
        # do some processing
        | 'my_function' >> beam.ParDo(my_function())
    )